<a href="https://colab.research.google.com/github/chaudhayabdullah786/Billing-Management-System/blob/main/Muhammad_Abdullah_231980077_Assignment_1_ML_for_Data_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 December 14,2025   

 Machine Learning FOr Data
 Science  

Sunday                                                                 Assignment 1

Muhammad Abdullah

231980077

Shawaiz Ali

231980079 **bold text**

Question#1
To prepare the raw data for k-NN classification,. Each file’s data was divided into segments of
1000 samples, and labels were assigned based on the motor condition (e.g., healthy or inner-race
fault). I then combined all the processed data into a single CSV file. Missing values were handled
by removing rows with null entries. As a result, I obtained a clean dataset containing 5168 rows
and 1001 columns, with 1000 feature columns and 1 label column.
The code does the following:
1. Explains its purpose.
2. Details the key steps involved.
# 3. Summarizes the output or final result. **bold text**

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Change folder_path to a Colab accessible location (e.g., /content/)
# You need to upload your CSV files to this location or mount Google Drive.
folder_path = Path(r"/content/") # Assuming files are uploaded directly to /content/

# Create an empty list to store DataFrames for each file
output_dfs = []
# Loop through all CSV files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file: {file_path}") # Added print statement to show the file being processed
        # Extract the file name without extension
        file_name_without_extension = file_name.split("-")[0]
        # Read the CSV file
        df = pd.read_csv(file_path)


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Folder containing CSVs
# IMPORTANT: Change this to a path accessible in Google Colab (e.g., /content/ or a mounted Google Drive path)
folder_path = Path(r"/content/") # Example: Assuming your CSV files are uploaded to /content/

output_dfs = []

# Loop through CSV files
for file_path in folder_path.glob("*.csv"):
    # Skip generated output files to prevent errors
    if file_path.name == "output.csv" or file_path.name == "distance_matrix.csv":
        print(f"Skipping generated file: {file_path.name}")
        continue

    print("Processing:", file_path.name)
    df = pd.read_csv(file_path, low_memory=False, on_bad_lines='skip')

    file_name_without_extension = file_path.stem

    # Extract 'Current-A' column (strip leading spaces)
    column_name_candidates = [col for col in df.columns if col.strip() == 'Current-A']
    if not column_name_candidates:
        print(f"Warning: ' Current-A' column not found in {file_path.name}. Skipping file.")
        continue
    column_name = column_name_candidates[0]
    current_a_data = df[column_name]

    # Chunk size
    chunk_size = 1000

    # Number of complete chunks
    num_complete_chunks = len(current_a_data) // chunk_size

    # Lists to store chunks and labels
    transposed_chunks_list = []
    labels_list = []

    # Process chunks
    for i in range(0, num_complete_chunks * chunk_size, chunk_size):
        chunk = current_a_data[i:i + chunk_size]

        # Transpose chunk
        transposed_chunk = chunk.values.reshape(1, -1)

        # Append chunk and label
        transposed_chunks_list.append(transposed_chunk)
        labels_list.append(file_name_without_extension)

    # Concatenate chunks
    output_array = np.concatenate(transposed_chunks_list, axis=0)

    # Create DataFrame
    output_df = pd.DataFrame(
        data=output_array,
        columns=[f'Value_{j}' for j in range(output_array.shape[1])]
    )

    # Add label column
    output_df['Label'] = labels_list

    # Append to main list
    output_dfs.append(output_df)

# Combine all CSVs
final_output_df = pd.concat(output_dfs, axis=0, ignore_index=True)

# Save final CSV
final_output_file_path = folder_path / "output.csv"
final_output_df.to_csv(final_output_file_path, index=False)

print("Saved final output to:", final_output_file_path)


In [ ]:
# Print a message to indicate that the final file has been saved
print('Final transposed data with labels saved to {}'.
 format(final_output_file_path))

In [ ]:
import pandas as pd
from pathlib import Path

# Full path to your output CSV
# Update this to the correct path in Colab, which is /content/output.csv
output_file_path = Path(r"/content/output.csv")

# Read CSV
df = pd.read_csv(output_file_path)

# Check for null values
null_values = df.isnull().sum()
print("Null Values:\n", null_values)


In [ ]:
df = df.dropna(axis = 0)
print(df.isnull().sum(axis = 0))

The dataset is clean and complete, with no missing values.

No imputation, row removal, or preprocessing for missing data is required.

This is ideal for machine learning models such as KNN, since distance-based algorithms are sensitive to missing values.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Full path to the CSV
# Corrected path to point to the Colab-accessible output file
output_file_path = Path(r"/content/output.csv")

# Read the CSV
transposed_data_combined = pd.read_csv(output_file_path)

# Remove the Label column
numerical_columns = transposed_data_combined.drop(columns=['Label'])

# Calculate the Euclidean distance matrix
distance_matrix = np.sqrt(
    np.maximum(0.0,
        (np.abs(numerical_columns.values)**2).sum(axis=1)[:, None] +
        (np.abs(numerical_columns.values)**2).sum(axis=1)[None, :] -
        2 * np.abs(numerical_columns.values).dot(np.abs(numerical_columns.values).T)
    )
)

# Convert to DataFrame
df_distance_matrix = pd.DataFrame(
    distance_matrix,
    index=numerical_columns.index,
    columns=numerical_columns.index
)

# Print distance matrix
print(df_distance_matrix)

# Save distance matrix to CSV
distance_file_path = output_file_path.parent / "distance_matrix.csv"
df_distance_matrix.to_csv(distance_file_path, index=False)

print("Saved distance matrix to:", distance_file_path)


The distance matrix contains pairwise Euclidean distances between all data samples. Zero values along the diagonal indicate self-distance, while non-zero values represent similarity differences between samples. This matrix forms the basis for neighbor selection in the K-NN classification algorithm.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Example usage
X = np.random.rand(100, 5)
y = np.random.randint(0, 2, 100)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)


The dataset has been split into training and testing sets.

Training set shape (80, 5): 80 samples used to train the model, each with 5 features.

Testing set shape (20, 5): 20 samples reserved for evaluating model performance on unseen data.

This split helps assess how well the model generalizes and reduces the risk of overfitting.

In [ ]:
class KNNClassifier:
   def __init__(self, k=3):
       self.k = k
       self.X_train = None
       self.y_train = None
   def fit(self, X_train, y_train):
       self.X_train = X_train
       self.y_train = y_train
   def predict(self, X_test):
       predictions = []
       for x_test_instance in X_test:
           distances = [self.euclidean_distance(x_test_instance,
   x_train_instance) for x_train_instance in self.X_train]
           nearest_indices = np.argsort(distances)[:self.k]
           nearest_labels = [self.y_train[idx] for idx in nearest_indices]
           predicted_label = self.most_common(nearest_labels)
           predictions.append(predicted_label)
       return predictions
   def euclidean_distance(self, x1, x2):
       return np.sqrt(np.sum((x1 - x2) ** 2))
   def most_common(self, lst):
       unique, counts = np.unique(lst, return_counts=True)
       index = np.argmax(counts)
       return unique[index]
# Assuming df is your DataFrame
X = df.iloc[:, :-1].values # Assuming the last column is the target variable
y = df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
  random_state=42)
knn_classifier = KNNClassifier(k=5)
knn_classifier.fit(X_train, y_train)
# Make predictions on the test set
predictions = knn_classifier.predict(X_test)
# Display the predictions
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Re-read the output.csv to ensure df is updated with all labels
output_file_path = Path(r"/content/output.csv")
df = pd.read_csv(output_file_path)

# Print unique labels and their counts from the 'Label' column
print("Unique labels and their counts in the dataset:")
print(df['Label'].value_counts())

print("\nTotal number of unique labels:", len(df['Label'].unique()))

In [ ]:
import random
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
def hold_out_split(X, y, test_size=0.2, random_seed=None):
    if random_seed:
       random.seed(random_seed)
    data = list(zip(X, y))
    random.shuffle(data)
    split_index = int(len(data) * (1 - test_size))
    train_data, test_data = data[:split_index], data[split_index:]
    X_train, y_train = zip(*train_data)
    X_test, y_test = zip(*test_data)
    return X_train, X_test, y_train, y_test
# Usage
X_train, X_test, y_train, y_test = hold_out_split(X, y, test_size=0.2,random_seed=42)
knn_classifier = KNNClassifier(k=5)
knn_classifier.fit(X_train, y_train)
# Make predictions on the test set
predictions = knn_classifier.predict(X_test)
# Display the predictions
print(len(predictions))
print(len(y_test))
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score,f1_score, confusion_matrix
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
# Calculate recall
recall = recall_score(y_test, predictions,average='weighted')
# Calculate precision
precision = precision_score(y_test, predictions,average='weighted')
# Calculate F1 Score
f1 = f1_score(y_test, predictions,average='weighted')
print(f"Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1 Score: {f1:.4f}")

In [ ]:
#y
import pandas as pd

duplicates = pd.merge(pd.DataFrame(X_train), pd.DataFrame(X_test), how='inner')
print("Duplicate rows between train and test:", len(duplicates))



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)


In [ ]:
print("Unique labels in y:", np.unique(y))


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
#y

In [ ]:
def custom_k_fold_validation(X, y, k=2):
    data = list(zip(X, y))
    np.random.shuffle(data)
    fold_size = len(data) // k
    folds = [data[i:i + fold_size] for i in range(0, len(data), fold_size)]
    for i in range(k):
        test_data = folds[i]
        train_data = [item for j in range(k) if j != i for item in folds[j]]
        X_train, y_train = zip(*train_data)
        X_test, y_test = zip(*test_data)
        yield np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)
# Example usage:
accuracies = []
for X_train, X_test, y_train, y_test in custom_k_fold_validation(X, y,2):
    knn_classifier = KNNClassifier(k=5)
    knn_classifier.fit(X_train, y_train)
# Make predictions on the test set
    predictions = knn_classifier.predict(X_test)
# Display the predictions
    accuracy = accuracy_score(y_test, predictions)
    accuracies.append(accuracy)
    print(accuracy)
overall_accuracy = np.mean(accuracies)
print(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Define the range of K values to try
k_values = range(1, 20, 2)

# Lists to store cross-validation and test accuracies
cv_accuracies = []
test_accuracies = []

# Perform grid search
for k in k_values:
    # Lists to store fold accuracies during cross-validation
    fold_accuracies = []
    print("Evaluating k =", k)

    # Cross-validation loop
    for X_train_fold, X_test_fold, y_train_fold, y_test_fold in custom_k_fold_validation(X_train, y_train, k=2):
        # Initialize KNN classifier
        knn_classifier = KNNClassifier(k=k)
        knn_classifier.fit(X_train_fold, y_train_fold)

        # Make predictions on the fold test set
        predictions_fold = knn_classifier.predict(X_test_fold)

        # Calculate accuracy for this fold
        fold_accuracy = accuracy_score(y_test_fold, predictions_fold)
        print("Fold accuracy:", fold_accuracy)
        fold_accuracies.append(fold_accuracy)

    # Calculate mean cross-validation accuracy for this K
    cv_accuracy = np.mean(fold_accuracies)
    cv_accuracies.append(cv_accuracy)

    # Train the model on the entire training set with the current K
    knn_classifier = KNNClassifier(k=k)
    knn_classifier.fit(X_train, y_train)

    # Make predictions on the test set
    predictions_test = knn_classifier.predict(X_test)

    # Calculate test accuracy for this K
    test_accuracy = accuracy_score(y_test, predictions_test)
    test_accuracies.append(test_accuracy)

# Find the best K
best_k = k_values[np.argmax(cv_accuracies)]
best_accuracy = cv_accuracies[np.argmax(cv_accuracies)]
print(f"Best K: {best_k}, Cross-validation Accuracy: {best_accuracy:.4f}")

# Plot the results
plt.plot(k_values, cv_accuracies, label='Cross-validation Accuracy')
plt.plot(k_values, test_accuracies, label='Test Accuracy')
plt.scatter([best_k], [best_accuracy], color='red', marker='o', label=f'Best K={best_k}')
plt.xlabel('K Value')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Assuming df is your DataFrame
X = df.iloc[:, :-1].values  # Features
labels = df.iloc[:, -1].values

# Handle any potential NaNs in X before standardization/PCA
# Re-apply dropna just in case df was loaded uncleanly elsewhere.
df_cleaned = df.dropna(axis=0) # Create a cleaned version
X_cleaned = df_cleaned.iloc[:, :-1].values
labels_cleaned = df_cleaned.iloc[:, -1].values

# Standardize the data
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X_cleaned)

# Determine max possible components
max_components = min(X_standardized.shape)  # min(n_samples, n_features)
n_components = min(400, max_components)    # adjust automatically

# Apply PCA
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_standardized)

# Display the result
print("Original shape:", X_cleaned.shape)
print("PCA transformed shape:", X_pca.shape)


In [ ]:
import pandas as pd
# Use labels_cleaned which corresponds to the cleaned X_pca data
labels_pca = pd.DataFrame(labels_cleaned)
df = pd.DataFrame(X_pca)
df = pd.concat([df, labels_pca], axis=1)
df

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KNeighborsClassifier
class KNNClassifier:

    def __init__(self, k=3):
        self.k = k
        self.model = KNeighborsClassifier(n_neighbors=self.k)
    def fit(self, X, y):
        self.model.fit(X, y)
        return self
    def predict(self, X):
        return self.model.predict(X)
    def score(self, X, y):
        return self.model.score(X, y)
    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))
    def most_common(self, lst):
        unique, counts = np.unique(lst, return_counts=True)
        index = np.argmax(counts)
        return unique[index]
# Assuming df is your DataFrame
# Ensure df is cleaned of any NaNs before splitting for the classifier
df_cleaned_for_knn = df.dropna(axis=0)
X = df_cleaned_for_knn.iloc[:, :-1].values # Assuming the last column is the target variable
y = df_cleaned_for_knn.iloc[:, -1].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                random_state=42)
knn_classifier = KNNClassifier(k=5)
knn_classifier.fit(X_train, y_train)
# Make predictions on the test set
predictions = knn_classifier.predict(X_test)
# Display the predictions
print("Number of predictions:", len(predictions))
print("Number of test samples:", len(X_test))
print("Predictions:", predictions)
# Calculate and display accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
X_train, X_test, y_train, y_test = hold_out_split(X, y, test_size=0.2,
                                                random_seed=42)
knn_classifier = KNNClassifier(k=5)
knn_classifier.fit(X_train, y_train)
# Make predictions on the test set
predictions = knn_classifier.predict(X_test)
# Display the predictions
print(len(predictions))
print(len(y_test))
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
accuracies = []
for X_train, X_test, y_train, y_test in custom_k_fold_validation(X, y,2):
    knn_classifier = KNNClassifier(k=5)
    knn_classifier.fit(X_train, y_train)
# Make predictions on the test set
    predictions = knn_classifier.predict(X_test)
# Display the predictions
# print(predictions)
    accuracy = accuracy_score(y_test, predictions)
    accuracies.append(accuracy)
    print(accuracy)
overall_accuracy = np.mean(accuracies)
print(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score,f1_score, confusion_matrix
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
# Calculate recall
recall = recall_score(y_test, predictions,average='weighted')
# Calculate precision
precision = precision_score(y_test, predictions,average='weighted')
# Calculate F1 Score
f1 = f1_score(y_test, predictions,average='weighted')
print(f"Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:4f}, F1 Score: {f1:.4f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KNeighborsClassifier

# Redefine KNNClassifier to be scikit-learn compatible
class KNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, k=3):
        self.k = k
        # Initialize the scikit-learn KNeighborsClassifier here
        self.model = KNeighborsClassifier(n_neighbors=self.k)

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        return self.model.score(X, y)

    # These methods are from the original custom class, but not strictly needed
    # for sklearn compatibility for cross_val_score.
    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def most_common(self, lst):
        unique, counts = np.unique(lst, return_counts=True)
        index = np.argmax(counts)
        return unique[index]

# Define the range of K values to try
k_values = range(1, 3)
# Lists to store cross-validation and test accuracies
cv_accuracies = []
test_accuracies = []
# Perform grid search
for k in k_values:
# Create a KNN classifier with the current K
   knn_classifier = KNNClassifier(k=k)
# Cross-validation using scikit-learn's KFold
   kf = KFold(n_splits=2, shuffle=True, random_state=42)
   fold_accuracies = cross_val_score(knn_classifier, X_train, y_train, cv=kf,scoring='accuracy')
# Calculate mean cross-validation accuracy for this K
   cv_accuracy = np.mean(fold_accuracies)
   cv_accuracies.append(cv_accuracy)

# Fit the model on the entire training set with the current K
   knn_classifier.fit(X_train, y_train)
# Make predictions on the test set
   predictions_test = knn_classifier.predict(X_test)
# Calculate test accuracy for this K
   test_accuracy = accuracy_score(y_test, predictions_test)
   test_accuracies.append(test_accuracy)
best_k = k_values[np.argmax(cv_accuracies)]
# Plot the results
plt.plot(k_values, cv_accuracies, label='Cross-validation Accuracy')
plt.plot(k_values, test_accuracies, label='Test Accuracy')
plt.scatter(best_k, max(cv_accuracies), color='red', marker='*', label=f'Best K = {best_k}')
plt.xlabel('K Value')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


### Question 2: K-NN with PCA-transformed data

Now, we will apply the K-NN algorithm to the dataset after applying Principal Component Analysis (PCA) for dimensionality reduction. We will repeat all the steps from Question 1.

In [ ]:
# Re-extract features (X) and labels (y) from the PCA-transformed DataFrame
X_pca_transformed = df.iloc[:, :-1].values # All columns except the last one as features
y_pca_transformed = df.iloc[:, -1].values  # The last column as labels

print("Shape of X after PCA:", X_pca_transformed.shape)
print("Unique labels after PCA:", np.unique(y_pca_transformed))

# Split data into training and testing sets using the PCA-transformed data
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(
    X_pca_transformed, y_pca_transformed, test_size=0.2, random_state=42, stratify=y_pca_transformed
)

print("Train shape (PCA):", X_train_pca.shape, "Test shape (PCA):", X_test_pca.shape)

#### K-NN Model Implementation (without built-in libraries) on PCA data

In [ ]:
# Initialize and train the custom KNN classifier with PCA data
knn_classifier_pca = KNNClassifier(k=5) # Using k=5 as an initial value
knn_classifier_pca.fit(X_train_pca, y_train_pca)

# Make predictions on the test set (PCA data)
predictions_pca = knn_classifier_pca.predict(X_test_pca)

# Calculate and display accuracy for PCA data
accuracy_pca = accuracy_score(y_test_pca, predictions_pca)
print(f"Accuracy with custom KNN (PCA data): {accuracy_pca * 100:.2f}%")

#### Apply hold-out validation on PCA data

In [ ]:
# Apply hold-out split with PCA data
X_train_holdout_pca, X_test_holdout_pca, y_train_holdout_pca, y_test_holdout_pca = hold_out_split(
    X_pca_transformed, y_pca_transformed, test_size=0.2, random_seed=42
)

# Initialize and train KNN classifier with hold-out split PCA data
knn_classifier_holdout_pca = KNNClassifier(k=5)
knn_classifier_holdout_pca.fit(X_train_holdout_pca, y_train_holdout_pca)

# Make predictions on the hold-out test set (PCA data)
predictions_holdout_pca = knn_classifier_holdout_pca.predict(X_test_holdout_pca)

# Calculate and display accuracy for hold-out split PCA data
accuracy_holdout_pca = accuracy_score(y_test_holdout_pca, predictions_holdout_pca)
print(f"Accuracy with hold-out split (PCA data): {accuracy_holdout_pca * 100:.2f}%")

#### Apply custom K-Fold cross-validation on PCA data

In [ ]:
accuracies_cv_pca = []
for X_train_fold_pca, X_test_fold_pca, y_train_fold_pca, y_test_fold_pca in custom_k_fold_validation(
    X_pca_transformed, y_pca_transformed, k=2
):
    knn_classifier_cv_pca = KNNClassifier(k=5)
    knn_classifier_cv_pca.fit(X_train_fold_pca, y_train_fold_pca)
    predictions_cv_pca = knn_classifier_cv_pca.predict(X_test_fold_pca)
    accuracy_cv_fold_pca = accuracy_score(y_test_fold_pca, predictions_cv_pca)
    accuracies_cv_pca.append(accuracy_cv_fold_pca)
    print(f"Fold Accuracy (PCA): {accuracy_cv_fold_pca}")

overall_accuracy_cv_pca = np.mean(accuracies_cv_pca)
print(f"Overall K-Fold Accuracy (PCA data): {overall_accuracy_cv_pca * 100:.2f}%")

#### Calculate classification measures (Accuracy, Recall, Precision, F1 Score) on PCA data

In [ ]:
# Using the predictions from the initial train_test_split with PCA data
# Calculate accuracy
accuracy_pca_metrics = accuracy_score(y_test_pca, predictions_pca)
# Calculate recall
recall_pca = recall_score(y_test_pca, predictions_pca, average='weighted')
# Calculate precision
precision_pca = precision_score(y_test_pca, predictions_pca, average='weighted')
# Calculate F1 Score
f1_pca = f1_score(y_test_pca, predictions_pca, average='weighted')

print(f"Metrics for PCA data:\nAccuracy: {accuracy_pca_metrics:.4f}, Recall: {recall_pca:.4f}, Precision: {precision_pca:.4f}, F1 Score: {f1_pca:.4f}")


#### Find the optimized K value and plot cross-validation and test accuracy (PCA data)

In [ ]:
# Define the range of K values to try (adjust as needed based on insights)
k_values_pca = range(1, 20, 2) # Example range, consider expanding if needed

cv_accuracies_pca = []
test_accuracies_pca = []

for k_val in k_values_pca:
    fold_accuracies_grid_pca = []
    print(f"Evaluating k = {k_val} (PCA data)")

    for X_train_fold_grid_pca, X_test_fold_grid_pca, y_train_fold_grid_pca, y_test_fold_grid_pca in custom_k_fold_validation(
        X_train_pca, y_train_pca, k=2 # Using X_train_pca for CV as per original problem setup
    ):
        knn_classifier_grid_pca = KNNClassifier(k=k_val)
        knn_classifier_grid_pca.fit(X_train_fold_grid_pca, y_train_fold_grid_pca)
        predictions_fold_grid_pca = knn_classifier_grid_pca.predict(X_test_fold_grid_pca)
        fold_accuracy_grid_pca = accuracy_score(y_test_fold_grid_pca, predictions_fold_grid_pca)
        fold_accuracies_grid_pca.append(fold_accuracy_grid_pca)

    cv_accuracy_mean_pca = np.mean(fold_accuracies_grid_pca)
    cv_accuracies_pca.append(cv_accuracy_mean_pca)

    knn_classifier_test_pca = KNNClassifier(k=k_val)
    knn_classifier_test_pca.fit(X_train_pca, y_train_pca)
    predictions_test_grid_pca = knn_classifier_test_pca.predict(X_test_pca)
    test_accuracy_grid_pca = accuracy_score(y_test_pca, predictions_test_grid_pca)
    test_accuracies_pca.append(test_accuracy_grid_pca)

best_k_pca = k_values_pca[np.argmax(cv_accuracies_pca)]
best_accuracy_pca = cv_accuracies_pca[np.argmax(cv_accuracies_pca)]
print(f"Best K (PCA data): {best_k_pca}, Cross-validation Accuracy: {best_accuracy_pca:.4f}")

plt.figure(figsize=(10, 6))
plt.plot(k_values_pca, cv_accuracies_pca, label='Cross-validation Accuracy (PCA)')
plt.plot(k_values_pca, test_accuracies_pca, label='Test Accuracy (PCA)')
plt.scatter([best_k_pca], [best_accuracy_pca], color='red', marker='o', label=f'Best K (PCA)={best_k_pca}')
plt.xlabel('K Value')
plt.ylabel('Accuracy')
plt.title('K-NN Accuracy vs. K Value (PCA-transformed Data)')
plt.legend()
plt.grid(True)
plt.show()

**Question#3**

Which model gives the best accuracy and why?
Using PCA reduced the number of features from 1000 to 400, which made the computations faster
and less complex. While the accuracy only improved slightly (by about 0.88%), reducing the
number of features helped avoid the curse of dimensionality. This means that even with fewer
features, the calculations were quicker, and the overall classification accuracy wasn’t significantly
affected.
Best Model The best-performing k-NN model used PCA and had
k=3. This combination achieved an accuracy of 17.71%

**Why It Performed Better:**

**• Reducing Dimensions:** PCA reduced the original 1000 features to just 400, minimizing
the impact of the curse of dimensionality. It likely eliminated irrelevant or noisy features

 **Better Generalization:** With fewer features, the model became less prone to overfitting,
improving its ability to perform well on unseen data.

 **Faster Computation:** Byloweringthedimensionality, PCA simplified distance calculations,
making the classification process quicker and more efficient

** Improved Class Separation:** PCA restructured the feature space to focus on the most
significant variations, which might have made it easier to distinguish between overlapping
classes.